# UIDAI Biometric Data Analysis file 1

In [117]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [118]:
df = pd.read_csv('/content/bio_cleaned_final.csv')

In [119]:
df.shape

(1408318, 7)

In [120]:
df.head()

,Unnamed: 0,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,0,01-03-2025,haryana,mahendragarh,123029,280,577
1,1,01-03-2025,bihar,madhepura,852121,144,369
2,2,01-03-2025,bihar,bhojpur,802158,256,980
3,3,01-03-2025,tamil nadu,madurai,625514,271,815
4,4,01-03-2025,maharashtra,ratnagiri,416702,155,529


In [123]:
df.drop(['Unnamed: 0'],inplace=True,axis=1)

In [ ]:
df['district'] = df['district'].str.upper()
df['state'] = df['state'].str.upper()

In [ ]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,UTTAR PRADESH,GORAKHPUR,273213,49,529
1,01-03-2025,ANDHRA PRADESH,CHITTOOR,517132,22,375
2,01-03-2025,GUJARAT,RAJKOT,360006,65,765
3,01-03-2025,ANDHRA PRADESH,SRIKAKULAM,532484,24,314
4,01-03-2025,RAJASTHAN,UDAIPUR,313801,45,785


In [ ]:
import json
import pandas as pd

# Load the pincode reference JSON file
with open('pin-code-data.json', 'r') as f:
    pincode_reference = json.load(f)

# Convert pincode values to strings for consistent lookup
df['pincode_str'] = df['pincode'].astype(str)

# Use map with default None for missing values
df['state_new'] = df['pincode_str'].map(
    lambda x: pincode_reference.get(x, {}).get('state', None)
)
df['district_new'] = df['pincode_str'].map(
    lambda x: pincode_reference.get(x, {}).get('district', None)
)

# Optional: Drop the temporary pincode_str column if not needed
df = df.drop('pincode_str', axis=1)

print(df.head())


         date           state    district  pincode  demo_age_5_17  \
0  01-03-2025   UTTAR PRADESH   GORAKHPUR   273213             49   
1  01-03-2025  ANDHRA PRADESH    CHITTOOR   517132             22   
2  01-03-2025         GUJARAT      RAJKOT   360006             65   
3  01-03-2025  ANDHRA PRADESH  SRIKAKULAM   532484             24   
4  01-03-2025       RAJASTHAN     UDAIPUR   313801             45   

   demo_age_17_       state_new district_new  
0           529   UTTAR PRADESH    GORAKHPUR  
1           375  ANDHRA PRADESH     CHITTOOR  
2           765         GUJARAT       RAJKOT  
3           314  ANDHRA PRADESH   SRIKAKULAM  
4           785       RAJASTHAN      UDAIPUR  


In [ ]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,state_new,district_new
0,01-03-2025,UTTAR PRADESH,GORAKHPUR,273213,49,529,UTTAR PRADESH,GORAKHPUR
1,01-03-2025,ANDHRA PRADESH,CHITTOOR,517132,22,375,ANDHRA PRADESH,CHITTOOR
2,01-03-2025,GUJARAT,RAJKOT,360006,65,765,GUJARAT,RAJKOT
3,01-03-2025,ANDHRA PRADESH,SRIKAKULAM,532484,24,314,ANDHRA PRADESH,SRIKAKULAM
4,01-03-2025,RAJASTHAN,UDAIPUR,313801,45,785,RAJASTHAN,UDAIPUR


In [ ]:
df.columns

Index(['date', 'state', 'district', 'pincode', 'demo_age_5_17', 'demo_age_17_',
       'state_new', 'district_new'],
      dtype='object')

In [124]:

df['state']=df['state'].str.lower()
df['district']=df['district'].str.lower()

In [125]:
df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,01-03-2025,haryana,mahendragarh,123029,280,577
1,01-03-2025,bihar,madhepura,852121,144,369
2,01-03-2025,bihar,bhojpur,802158,256,980
3,01-03-2025,tamil nadu,madurai,625514,271,815
4,01-03-2025,maharashtra,ratnagiri,416702,155,529


In [130]:
df.shape

(1408318, 6)

In [128]:
df_pre_final = df.copy()

In [131]:
df_pre_final.shape

(1408318, 6)

In [132]:
df_pre_final.columns

Index(['date', 'state', 'district', 'pincode', 'bio_age_5_17', 'bio_age_17_'], dtype='object')

In [133]:
import pandas as pd
import json

# Load your pincode JSON file
with open('pin-code-data.json', 'r') as f:
    pincode_data = json.load(f)

# Create a mapping of state to its valid pincodes
state_to_pincodes = {}
for pincode, info in pincode_data.items():
    state = info['state'].upper()

    if state not in state_to_pincodes:
        state_to_pincodes[state] = set()
    state_to_pincodes[state].add(pincode)

# Function to check if pincode belongs to state
def check_pincode_in_state(pincode, state):
    """
    Check if pincode belongs to the given state
    Returns: True if pincode exists for that state, False otherwise
    """
    pincode_str = str(pincode).strip()
    state_upper = state.upper().strip()

    if state_upper in state_to_pincodes:
        return pincode_str in state_to_pincodes[state_upper]
    else:
        return False

# Create new 'match' column

df_pre_final['match'] = df_pre_final.apply(
    lambda row: check_pincode_in_state(row['pincode'], row['state']),
    axis=1
)

# Display results
print(df_pre_final[['state', 'pincode', 'match']])

# Get summary
print("\nMatch Summary:")
print(df_pre_final['match'].value_counts())

# Show unmatched records
unmatched = df_pre_final[df_pre_final['match'] == False]
print(f"\nUnmatched Records: {len(unmatched)}")
print(unmatched[['state', 'pincode', 'match']])


               state  pincode  match
0            haryana   123029   True
1              bihar   852121   True
2              bihar   802158   True
3         tamil nadu   625514   True
4        maharashtra   416702   True
...              ...      ...    ...
1408313  west bengal   743384  False
1408314  west bengal   743392  False
1408315  west bengal   743502   True
1408316  west bengal   733201   True
1408317  west bengal   733213  False

[1408318 rows x 3 columns]

Match Summary:
match
True     1359107
False      49211
Name: count, dtype: int64

Unmatched Records: 49211
                 state  pincode  match
19       uttar pradesh   225408  False
80               bihar   852210  False
136              bihar   800018  False
169        west bengal   733252  False
179      uttar pradesh   241501  False
...                ...      ...    ...
1408298    west bengal   721665  False
1408299    west bengal   721669  False
1408313    west bengal   743384  False
1408314    west bengal   74339

In [146]:
df_one_save = df_pre_final[df_pre_final['match']==False]

In [148]:
df_one_save.to_csv('first_false_bio.csv',index=False)

In [134]:
df_pre_clean = df_pre_final[df_pre_final['match'] == True]

In [138]:
df_pre_clean.shape

(1359107, 8)

In [ ]:
df_one_save.to_csv('first_flase_demo.csv')

In [137]:
df_pre_final.shape

(1408318, 7)

In [136]:
import pandas as pd
import json

# Load your pincode JSON file
with open('pin-code-data.json', 'r') as f:
    pincode_data = json.load(f)

# Create a mapping of state to its valid pincodes
district_to_pincodes = {}
for pincode, info in pincode_data.items():
    district = info['district'].upper()

    if district not in district_to_pincodes:
        district_to_pincodes[district] = set()
    district_to_pincodes[district].add(pincode)

# Function to check if pincode belongs to state
def check_pincode_in_district(pincode, district):
    """
    Check if pincode belongs to the given state
    Returns: True if pincode exists for that state, False otherwise
    """
    pincode_str = str(pincode).strip()
    district_upper = district.upper().strip()

    if district_upper in district_to_pincodes:
        return pincode_str in district_to_pincodes[district_upper]
    else:
        return False

# Create new 'match' column

df_pre_clean['match_district'] = df_pre_clean.apply(
    lambda row: check_pincode_in_district(row['pincode'], row['district']),
    axis=1
)

# Display results
print(df_pre_clean[['district', 'pincode', 'match_district']])

# Get summary
print("\nMatch Summary:")
print(df_pre_clean['match_district'].value_counts())

# Show unmatched records
unmatched = df_pre_clean[df_pre_clean['match_district'] == False]
print(f"\nUnmatched Records: {len(unmatched)}")
print(unmatched[['district', 'pincode', 'match_district']])


                  district  pincode  match_district
0             mahendragarh   123029            True
1                madhepura   852121            True
2                  bhojpur   802158            True
3                  madurai   625514            True
4                ratnagiri   416702            True
...                    ...      ...             ...
1408310  south 24 parganas   743371           False
1408311  south 24 parganas   743375           False
1408312  south 24 parganas   743378           False
1408315  south 24 parganas   743502           False
1408316     uttar dinajpur   733201           False

[1359107 rows x 3 columns]

Match Summary:
match_district
True     1089548
False     269559
Name: count, dtype: int64

Unmatched Records: 269559
                  district  pincode  match_district
11       paschim medinipur   721147           False
26                   dahod   389382           False
29            vizianagaram   535592           False
31                  ra

/tmp/ipython-input-3169469129.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pre_clean['match_district'] = df_pre_clean.apply(


In [ ]:
df_pre_clean.to_csv('demo_completely_clean.csv')

In [143]:
df000 = df_pre_clean[df_pre_clean['match_district']==False]

In [144]:
df000.shape

(269559, 8)

In [141]:
df000.to_csv('bio_complete_clean.csv',index=False)

In [145]:
df000.to_csv('second_false_bio.csv',index=False)

In [ ]:
df_demo_final[df_demo_final['match']==False]

,date,state,district,pincode,demo_age_5_17,demo_age_17_,match
